In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
cms_data = pd.read_csv('CMS Prescription Data.csv') 
cms_data.rename(columns={"Prscrbr_NPI": "npi"},inplace = True)

gnrc_unique= cms_data['Gnrc_Name'].unique()

In [3]:
physicians_data = pd.read_csv('Ordering Physicians.csv') 

physicians_unique = physicians_data['NPI'].unique()

In [15]:
# data_model = pd.read_csv('data_model.csv') 


In [54]:
features = ['Prscrbr_NPI','Gnrc_Name','Tot_Clms', 'Tot_Day_Suply','GE65_Tot_Clms','GE65_Tot_Day_Suply']


### Reading the big data with chunksize to avoid low memory issues

In [2]:
# # Set the chunk size (in number of rows)
# chunk_size = 10000

# # Initialize an empty list to store the chunks
# chunks = []

# # Loop through the file in chunks and append each chunk to the list

# # reading specic columns only that will be used in machine learning model
# for chunk in pd.read_csv('Data.csv', chunksize=chunk_size,encoding='ISO-8859-1', low_memory=False):
#     chunks.append(chunk)

# # Concatenate the chunks into a single DataFrame
# df = pd.concat(chunks)

In [6]:
# prescriber_data = pd.read_csv('CMS Prescription Data.csv') 
# df.rename(columns={"Prscrbr_NPI": "npi"},inplace = True)

### Writing the data that will be used in model

In [9]:
# Set the file path and name
# file_path = 'data_model.csv'

# # Set the chunk size (in number of rows)
# chunk_size = 10000

# # Initialize a counter to track the number of rows written
# row_count = 0

# # Loop through the data in chunks and write each chunk to the file
# for i, chunk in enumerate(pd.read_csv('Data.csv', chunksize=chunk_size,encoding='ISO-8859-1', low_memory=False)):
#     if i == 0:
#         df.iloc[:chunk_size].to_csv(file_path, mode='w', index=False)
#     else:
#         df.iloc[row_count:row_count+chunk_size].to_csv(file_path, mode='a', header=False, index=False)
#     row_count += chunk_size

### Reading it again to get better memory usage than the bigger data

In [51]:
# Set the chunk size (in number of rows)
chunk_size = 10000

# Initialize an empty list to store the chunks
chunks = []

# Loop through the file in chunks and append each chunk to the list

# reading specic columns only that will be used in machine learning model
for chunk in pd.read_csv('data_model.csv', chunksize=chunk_size,encoding='ISO-8859-1', low_memory=False):
    chunks.append(chunk)

# Concatenate the chunks into a single DataFrame
data_model = pd.concat(chunks)

In [6]:
# Use the "isin" method to select rows where the values match the list
data_model = data_model[data_model['Gnrc_Name'].isin(gnrc_unique)]

In [7]:
features = ['Tot_Clms', 'Tot_Day_Suply','GE65_Tot_Clms','GE65_Tot_Day_Suply']

#Get unique generic names
unique_generic_names_prescribtion = data_model['Gnrc_Name'].unique()

# Group data by NPI and generic name, and count occurrences
counts = data_model.groupby(['npi', 'Gnrc_Name']).size().unstack(fill_value=0)

# Reindex with all unique generic names
generic_name_counts = counts.reindex(columns=unique_generic_names_prescribtion, fill_value=0)

# Reset index and rename columns
generic_name_counts.reset_index(inplace=True)
generic_name_counts.rename_axis(None, axis=1, inplace=True)


# group the original data by NPI and sum the total_claim_count and total_day_supply columns
summed_data = data_model.groupby('npi')[features].sum().reset_index()
prescribtion = pd.merge(generic_name_counts, summed_data, on='npi', how='inner')

In [10]:
# my_npi_list = [1467457358, 1174528129]
# my_physicians = prescribtion[prescribtion['npi'].isin(my_npi_list)]

903876

In [8]:
physicians_data = cms_data[cms_data['npi'].isin(physicians_unique)]

In [9]:
features = ['Tot_Clms', 'Tot_Day_Suply','GE65_Tot_Clms','GE65_Tot_Day_Suply']

#using the same unique generic name of the big data to maintain the order
unique_generic_names = unique_generic_names_prescribtion

# Group data by NPI and generic name, and count occurrences
counts = physicians_data.groupby(['npi', 'Gnrc_Name']).size().unstack(fill_value=0)

# Reindex with all unique generic names
generic_name_counts = counts.reindex(columns=unique_generic_names, fill_value=0)

# Reset index and rename columns
generic_name_counts.reset_index(inplace=True)
generic_name_counts.rename_axis(None, axis=1, inplace=True)


# group the original data by NPI and sum the total_claim_count and total_day_supply columns
summed_data = physicians_data.groupby('npi')[features].sum().reset_index()
prescribtion_physicians = pd.merge(generic_name_counts, summed_data, on='npi', how='inner')

In [27]:
prescribtion_physicians.head()

,npi,Amiodarone Hcl,Amlodipine Besylate,Bumetanide,Apixaban,Hydralazine Hcl,Metoprolol Tartrate,Prednisone,Trazodone Hcl,Cyclobenzaprine Hcl,...,Bosentan,Tafamidis,Treprostinil Diolamine,Lapatinib Ditosylate,Dobutamine Hcl,Treprostinil Sodium,Tot_Clms,Tot_Day_Suply,GE65_Tot_Clms,GE65_Tot_Day_Suply
0,1013262419,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,128,6600,31.0,1965.0
1,1013379841,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,12,720,12.0,720.0
2,1013984012,1,1,0,1,1,1,1,0,0,...,0,0,0,0,0,0,1814,105434,914.0,51484.0
3,1043279326,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1323,66666,819.0,42090.0
4,1053492215,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,153,6518,37.0,1058.0


In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [12]:

# Define the features to use in the similarity calculation
features = ['total_claim_count', 'total_day_supply', 'total_drug_cost', 'generic_name_count']

Y = prescribtion['npi']
X = prescribtion.drop('npi', axis=1)

# Normalize the data
scaler = StandardScaler()
X  = scaler.fit_transform(X)


# Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0, random_state=42)

# Train the KNN model
k = 10
distance_metric = 'euclidean'
knn_model = KNeighborsClassifier(n_neighbors=k, metric=distance_metric)
knn_model.fit(X, Y)

# Evaluate the model



KNeighborsClassifier(metric='euclidean', n_neighbors=10)

In [13]:
final_columns = prescribtion_physicians.columns.insert(0,'physician_npi')

In [14]:
final_columns = list(final_columns)

In [15]:
final_columns[1] = "similar_npi"


In [16]:
final_columns

['physician_npi',
 'similar_npi',
 'Amiodarone Hcl',
 'Amlodipine Besylate',
 'Bumetanide',
 'Apixaban',
 'Hydralazine Hcl',
 'Metoprolol Tartrate',
 'Prednisone',
 'Trazodone Hcl',
 'Cyclobenzaprine Hcl',
 'Diclofenac Sodium',
 'Duloxetine Hcl',
 'Fentanyl',
 'Gabapentin',
 'Hydrocodone/Acetaminophen',
 'Morphine Sulfate',
 'Oxycodone Hcl',
 'Oxycodone Hcl/Acetaminophen',
 'Pregabalin',
 'Tizanidine Hcl',
 'Tramadol Hcl',
 'Amoxicillin',
 'Cephalexin',
 'Alendronate Sodium',
 'Fluconazole',
 'Sucralfate',
 'Albuterol Sulfate',
 'Allopurinol',
 'Alprazolam',
 'Amoxicillin/Potassium Clav',
 'Atenolol',
 'Atorvastatin Calcium',
 'Carvedilol',
 'Chlorthalidone',
 'Clopidogrel Bisulfate',
 'Donepezil Hcl',
 'Doxazosin Mesylate',
 'Enalapril Maleate',
 'Escitalopram Oxalate',
 'Ezetimibe',
 'Fenofibrate',
 'Fenofibrate Nanocrystallized',
 'Fluoxetine Hcl',
 'Fluticasone Propionate',
 'Furosemide',
 'Glipizide',
 'Hydrochlorothiazide',
 'Isosorbide Mononitrate',
 'Labetalol Hcl',
 'Levothyro

In [40]:
similar_physicians = []
k_most_similar = 20
results = pd.DataFrame(columns=final_columns)

for i in range(len(prescribtion_physicians)):
    row = prescribtion_physicians.iloc[i, :]
    
    npi = int(row['npi'])
    row_without_npi = row.drop('npi')
    row_without_npi = row_without_npi.values.reshape(1, -1)
    
    row_without_npi  = scaler.transform(row_without_npi)
    my_data_similarities = knn_model.kneighbors(row_without_npi, n_neighbors=k_most_similar, return_distance=False)
    most_similar = prescribtion.iloc[my_data_similarities.flatten()]
    most_similar['physician_npi'] = str(npi)
    most_similar['similar_npi'] = most_similar['npi']
    most_similar.drop('npi',axis=1,inplace=True)
    results = results.append(most_similar, ignore_index=True)

In [41]:
results.columns

Index(['physician_npi', 'similar_npi', 'Amiodarone Hcl', 'Amlodipine Besylate',
       'Bumetanide', 'Apixaban', 'Hydralazine Hcl', 'Metoprolol Tartrate',
       'Prednisone', 'Trazodone Hcl',
       ...
       'Bosentan', 'Tafamidis', 'Treprostinil Diolamine',
       'Lapatinib Ditosylate', 'Dobutamine Hcl', 'Treprostinil Sodium',
       'Tot_Clms', 'Tot_Day_Suply', 'GE65_Tot_Clms', 'GE65_Tot_Day_Suply'],
      dtype='object', length=231)

In [42]:
results.head()

,physician_npi,similar_npi,Amiodarone Hcl,Amlodipine Besylate,Bumetanide,Apixaban,Hydralazine Hcl,Metoprolol Tartrate,Prednisone,Trazodone Hcl,...,Bosentan,Tafamidis,Treprostinil Diolamine,Lapatinib Ditosylate,Dobutamine Hcl,Treprostinil Sodium,Tot_Clms,Tot_Day_Suply,GE65_Tot_Clms,GE65_Tot_Day_Suply
0,1013262419,1013262419,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,128,6600,31.0,1965.0
1,1013262419,1104955699,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,59,2730,11.0,330.0
2,1013262419,1093357386,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,80,4995,0.0,0.0
3,1013262419,1720354921,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,105,5440,52.0,2175.0
4,1013262419,1487095105,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,103,5220,13.0,390.0


In [44]:
# results.to_csv('similarities_result.csv')

In [3]:
similar_results = pd.read_csv('similarities_result.csv')

In [4]:
similar_results.head()

,Unnamed: 0,physician_npi,similar_npi,Amiodarone Hcl,Amlodipine Besylate,Bumetanide,Apixaban,Hydralazine Hcl,Metoprolol Tartrate,Prednisone,...,Bosentan,Tafamidis,Treprostinil Diolamine,Lapatinib Ditosylate,Dobutamine Hcl,Treprostinil Sodium,Tot_Clms,Tot_Day_Suply,GE65_Tot_Clms,GE65_Tot_Day_Suply
0,0,1013262419,1013262419,0,0,0,0,0,0,0,...,0,0,0,0,0,0,128,6600,31.0,1965.0
1,1,1013262419,1104955699,0,0,0,0,0,0,0,...,0,0,0,0,0,0,59,2730,11.0,330.0
2,2,1013262419,1093357386,0,0,0,0,0,0,0,...,0,0,0,0,0,0,80,4995,0.0,0.0
3,3,1013262419,1720354921,0,0,0,0,0,0,0,...,0,0,0,0,0,0,105,5440,52.0,2175.0
4,4,1013262419,1487095105,0,0,0,0,0,0,0,...,0,0,0,0,0,0,103,5220,13.0,390.0


In [5]:
unique_similar_npi = similar_results['similar_npi'].unique()

In [46]:
# similar_data = prescribtion[prescribtion['npi'].isin(unique_similar_npi)]

In [49]:
len(similar_data['npi'].unique())

1336

In [57]:
df.columns

Index(['Prscrbr_NPI', 'Prscrbr_Last_Org_Name', 'Prscrbr_First_Name',
       'Prscrbr_City', 'Prscrbr_State_Abrvtn', 'Prscrbr_State_FIPS',
       'Prscrbr_Type', 'Prscrbr_Type_Src', 'Brnd_Name', 'Gnrc_Name',
       'Tot_Clms', 'Tot_30day_Fills', 'Tot_Day_Suply', 'Tot_Drug_Cst',
       'Tot_Benes', 'GE65_Sprsn_Flag', 'GE65_Tot_Clms', 'GE65_Tot_30day_Fills',
       'GE65_Tot_Drug_Cst', 'GE65_Tot_Day_Suply', 'GE65_Bene_Sprsn_Flag',
       'GE65_Tot_Benes'],
      dtype='object')

In [6]:
similar_data = df[df['Prscrbr_NPI'].isin(unique_similar_npi)]

In [7]:
len(similar_data)

30160

In [8]:
# similar_data.to_csv('similar_cms_data.csv')